In [1]:
import torch,json,os,importlib

Download the pretrained models from https://zenodo.org/record/4282667 and extract the models directories into the "pretrained_models" directory.



In [2]:
os.listdir("pretrained_models/")

['sub1', 'README.md', 'sub3', 'ensem2', 'sub2', 'ensem3', 'ensem1']

# Select the pretrained model path

In [21]:
model_path="pretrained_models/sub1/"
model_path="pretrained_models/sub2/"
#model_path="pretrained_models/sub3/"

# Loading the model specified by model_path

In [4]:
os.listdir(model_path)

['dataset_config.json',
 'eval_predictions_probs.pth',
 'description.txt',
 'eval_predictions.csv',
 'model_from_training_state_dict.pth',
 'model_config.json',
 'model_state_dict.pth']

### Loading the model config from json

In [5]:
with open(model_path+'model_config.json',"r")as f:
    model_config=json.load(f)


### Making an instance

In [6]:
module = importlib.import_module('models.{}'.format(model_config['arch']))
Network = getattr(module, 'Network')
model=Network(model_config)

n_blocks_per_stage is specified ignoring the depth param, nc=None



Decomposed Conv2d with decomp_factor of  4 




### Loading the pretrained weights

In [7]:
state_dict=torch.load(model_path+"/model_state_dict.pth")
#state_dict=torch.load(model_path+"/model_from_training_state_dict.pth")


In [8]:
state_dict['in_c.0.weight']

tensor([[[[-0.0332, -0.0353, -0.0135, -0.0505, -0.0453],
          [-0.0552, -0.0347,  0.0185, -0.0019, -0.0548],
          [-0.0089,  0.0357,  0.0280,  0.0047,  0.0296],
          [ 0.0402,  0.0489, -0.0048, -0.0035, -0.0084],
          [-0.0181,  0.0472,  0.0243, -0.0381,  0.1149]],

         [[ 0.0471,  0.1449,  0.0569,  0.1116,  0.0880],
          [-0.0413, -0.1156, -0.0679, -0.0853, -0.1174],
          [-0.0305, -0.0082, -0.0980, -0.0265,  0.0358],
          [-0.0673, -0.0496,  0.0275,  0.0101, -0.0198],
          [-0.0550, -0.0758, -0.0014,  0.0543,  0.0133]]],


        [[[-0.0581, -0.0017,  0.0217, -0.0299, -0.0004],
          [-0.0473,  0.0263,  0.0417,  0.0317,  0.0562],
          [-0.0889, -0.0444, -0.0570, -0.0151, -0.0479],
          [ 0.0270,  0.0557, -0.0411,  0.0242,  0.0256],
          [-0.1002, -0.0568, -0.0278, -0.0842, -0.1010]],

         [[-0.0611, -0.0424, -0.0272, -0.0378,  0.0148],
          [ 0.0748,  0.0144,  0.0188,  0.1204,  0.1204],
          [-0.0231, -0.

In [9]:
# if the weights are float16 cast the model
if [v for k,v in state_dict.items()][0].dtype==torch.float16:
    model=model.half()
    print("model in float16")
model.load_state_dict(state_dict)


model in float16


<All keys matched successfully>

In [10]:
model=model.cuda()

### Parameters count


In [11]:
def count_non_zero_params_nonbn(model):
    sum_params = 0
    sum_non_zero= 0 
    desc=""
    def calc_params(model):
        nonlocal desc, sum_params, sum_non_zero
        skip="count"
        if "batchnorm" in  type(model).__name__.lower():
            skip="skip"
        for k,p in model.named_parameters(recurse=False):
            if p.requires_grad:
                nonzero = (p != 0).sum()
                total= p.numel()
                desc += f"type {type(model).__name__}, {k},  {total}, {nonzero}, {p.dtype}, {skip} " + "\n"
                if skip!="skip":
                    sum_params+=total
                    sum_non_zero+=nonzero
    model.apply(calc_params)       
    return desc,sum_params,sum_non_zero

In [12]:
desc,sum_params,sum_non_zero=count_non_zero_params_nonbn(model)
print("Total: ",sum_params, "Non-Zero:",sum_non_zero)
print(desc)

Total:  17520 Non-Zero: tensor(17520, device='cuda:0')
type Conv2d, weight,  2400, 2400, torch.float16, count 
type BatchNorm2d, weight,  48, 48, torch.float16, skip 
type BatchNorm2d, bias,  48, 48, torch.float16, skip 
type Conv2d, weight,  576, 576, torch.float16, count 
type Conv2d, weight,  1296, 1296, torch.float16, count 
type Conv2d, weight,  576, 576, torch.float16, count 
type BatchNorm2d, weight,  48, 48, torch.float16, skip 
type BatchNorm2d, bias,  48, 48, torch.float16, skip 
type Conv2d, weight,  576, 576, torch.float16, count 
type Conv2d, weight,  144, 144, torch.float16, count 
type Conv2d, weight,  576, 576, torch.float16, count 
type BatchNorm2d, weight,  48, 48, torch.float16, skip 
type BatchNorm2d, bias,  48, 48, torch.float16, skip 
type Conv2d, weight,  576, 576, torch.float16, count 
type Conv2d, weight,  1296, 1296, torch.float16, count 
type Conv2d, weight,  576, 576, torch.float16, count 
type BatchNorm2d, weight,  48, 48, torch.float16, skip 
type BatchNor

# Loading DataSet

In [13]:
import datasets
# the eval json config dataset specifies that the norms should be calculated from all the training clips. 
with open('configs/datasets/dcase2020b_eval.json',"r")as f:
    dataset_config=json.load(f)
datasetm=datasets.DatasetsManager(dataset_config['audiodataset'])

In [14]:
trainds=datasetm.get_train_set()
testds=datasetm.get_train_set()
evalds=datasetm.get_sub_dataset()

loading dataset from 'd20t1b'
normalizing dataset
normalized train!
attempting to load x from cache at datasets/cached_datasets/d20t1b/tr_mean_676d70_f1_apd18_stereo_obj.pt...
loaded datasets/cached_datasets/d20t1b/tr_mean_676d70_f1_apd18_stereo_obj.pt from cache in 0.0007960796356201172 
attempting to load x from cache at datasets/cached_datasets/d20t1b/tr_std_676d70_f1_apd18_stereo_obj.pt...
loaded datasets/cached_datasets/d20t1b/tr_std_676d70_f1_apd18_stereo_obj.pt from cache in 0.00045108795166015625 
normalized train!
normalized test!
normalized SUB dataset!
loading dataset from 'd20t1b_sub'


## Sanity checks

### dev train

In [16]:
all_y=0
corr_y=0
model.eval()
model.cuda()
half_mode=True
data_loader = torch.utils.data.DataLoader(trainds,
                                          batch_size=1,
                                          shuffle=False)
with torch.no_grad():
    for i,(x,idx,y) in enumerate(data_loader):
        if i%10==0:
            print(i,"/",len(data_loader),end="\r")
        x=x.cuda()
        if half_mode:
            x=x.half()
        out=model(x)
        all_y+=1
        corr_y+=(torch.argmax(out).cpu()==y).sum().item()
print("all ", all_y," correct ",corr_y," ratio=",corr_y*1./all_y) 
    

x: torch.Size([1, 2, 256, 431])
in_c: torch.Size([1, 48, 127, 215])
stage1: torch.Size([1, 48, 15, 26])
stage2: torch.Size([1, 48, 15, 26])
stage3: torch.Size([1, 48, 15, 26])
feed_forward: torch.Size([1, 3, 1, 1])
logit: torch.Size([1, 3])
all  14400  correct  14268  ratio= 0.9908333333333333


# eval

In [17]:
all_y=0
corr_y=0
model.eval()
model.cuda()
half_mode=True
data_loader = torch.utils.data.DataLoader(evalds,
                                          batch_size=1,
                                          shuffle=False,num_workers=10)
preds=[]
file_names=[]
with torch.no_grad():
    for i,(x,idx,y) in enumerate(data_loader):
        if i%10==0:
            print(i,"/",len(data_loader),end="\r")
        x=x.cuda()
        if half_mode:
            x=x.half()
        out=model(x)
        all_y+=1
        file_names+=[a for a in idx]
        preds+=[out.cpu()]
print("all ", all_y," correct ",corr_y," ratio=",corr_y*1./all_y) 
    

all  8640  correct  0  ratio= 0.0


In [18]:
pfiles,pprobs=torch.load(model_path+"eval_predictions_probs.pth")

In [19]:
pfiles==file_names

True

In [20]:
print("matching",(torch.argmax(pprobs.float(),1)== torch.argmax(torch.cat(preds,0).float(),1) ).sum(),"out of " ,pprobs.shape[0])


matching tensor(8640) out of  8640
